<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU25_Modelo de gestión de Lista de Espera Quirúrgica</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>09.1.- Data Cleansing - Basic</font>

<font color='steelblue'>Data Cleaning refers to identifying and correcting (or removing) errors in the dataset that may negatively impact a predictive model, replacing, modifying, or deleting the dirty or coarse data.</font>

## <font color='steelblue'>Tasks</font>

## Consideraciones casos CitizenLab programados en R

* La mayoría de las tareas de este proceso se han realizado en los notebooks del proceso 05 Data Collection porque eran necesarias para las tareas ETL. En esos casos, en este notebook se referencia al notebook del proceso 05 correspondiente
* Por tanto en los notebooks de este proceso de manera general se incluyen las comprobaciones necesarias, y comentarios si procede
* Las tareas del proceso se van a aplicar solo a los archivos que forman parte del despliegue, ya que hay muchos archivos intermedios que no procede pasar por este proceso
* El nombre de archivo del notebook hace referencia al nombre de archivo del proceso 05 al que se aplica este proceso, por eso pueden no ser correlativa la numeración
* Las comprobaciones se van a realizar teniendo en cuenta que el lenguaje utilizado en el despliegue de este caso es R

## <font color='green'>File</font>

- <font color='steelblue'> <b>Input File: CU_25_05_07_03_lista_espera_completo_split.csv</font>
- <font color='steelblue'> <b>Output File: No aplica</font>

## <font color='green'>Settings</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [140]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "LC_COLLATE=es_ES.UTF-8;LC_CTYPE=es_ES.UTF-8;LC_MONETARY=es_ES.UTF-8;LC_NUMERIC=C;LC_TIME=es_ES.UTF-8"

### Libraries to use

In [141]:
library(readr)
library(dplyr)
library(sf)
library(tidyr)
library(stringr)

### Paths

In [142]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if using this option

In [143]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [144]:
iFile <- "CU_25_08_01_lista_espera_completo_split.csv"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_25_08_01_lista_espera_completo_split.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [145]:
data <- read.csv(file_data)
colnames(data)

[1] "Hospital"          "Especialidad"      "total_pacientes"  
 [4] "ano"               "semana"            "CODCNH"           
 [7] "id_area"           "nombre_area"       "cmunicipio"       
[10] "Municipio"         "CAMAS"             "Clase"            
[13] "Dependencia"       "TAC"               "RM"               
[16] "GAM"               "HEM"               "ASD"              
[19] "LIT"               "BCO"               "ALI"              
[22] "SPECT"             "PET"               "MAMOS"            
[25] "DO"                "DIAL"              "X"                
[28] "Y"                 "t3_1"              "t1_1"             
[31] "t2_1"              "t2_2"              "t4_1"             
[34] "t4_2"              "t4_3"              "t5_1"             
[37] "t6_1"              "t7_1"              "t8_1"             
[40] "t9_1"              "t10_1"             "t11_1"            
[43] "t12_1"             "capacidad"         "pacientes"        
[46] "consultas"         "hospitalizaciones" "Target"           
[49] "is_train"

<font color='tomato'>Visualizo los datos.</font>

Estructura de  los datos:

In [146]:
data |> glimpse()

Rows: 55,680
Columns: 49
$ Hospital          <chr> "HOSPITAL REY JUAN CARLOS", "HOSPITAL CENTRAL DE LA …
$ Especialidad      <chr> "Urología", "Odontoestomatología", "Ginecología", "D…
$ total_pacientes   <int> 344, 0, 52, 37, 0, 4, 0, 718, 0, 271, 108, 0, 34, 86…
$ ano               <int> 2021, 2020, 2021, 2021, 2021, 2020, 2021, 2020, 2021…
$ semana            <int> 30, 36, 49, 23, 3, 5, 50, 7, 35, 1, 42, 10, 21, 33, …
$ CODCNH            <int> 281348, 280724, 281292, 281292, 281236, 280724, 2807…
$ id_area           <int> 8, 7, 11, 11, 11, 7, 3, 6, 1, 2, 2, 8, 11, 11, 1, 3,…
$ nombre_area       <chr> "Sur-Oeste I", "Centro-Oeste", "Sur Ii", "Sur Ii", "…
$ cmunicipio        <int> 280920, 280796, 280133, 280133, 281610, 280796, 2800…
$ Municipio         <chr> "Móstoles", "Madrid", "Aranjuez", "Aranjuez", "Valde…
$ CAMAS             <int> 382, 475, 98, 98, 182, 475, 507, 613, 269, 1143, 156…
$ Clase             <chr> "Hospitales Generales", "Hospitales Generales", "Hos…
$ Dependencia  

Muestra de los primeros datos:

In [147]:
data |> slice_head(n = 5)

Hospital,Especialidad,total_pacientes,ano,semana,CODCNH,id_area,nombre_area,cmunicipio,Municipio,⋯,t9_1,t10_1,t11_1,t12_1,capacidad,pacientes,consultas,hospitalizaciones,Target,is_train
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
HOSPITAL REY JUAN CARLOS,Urología,344,2021,30,281348,8,Sur-Oeste I,280920,Móstoles,⋯,0.3348780,0.1369254,0.5072726,0.5849309,17,1447,573,12,54.45,TRUE
HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,Odontoestomatología,0,2020,36,280724,7,Centro-Oeste,280796,Madrid,⋯,0.4914365,0.1217100,0.4915713,0.5597213,0,1211,45,0,0.00,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,Ginecología,52,2021,49,281292,11,Sur Ii,280133,Aranjuez,⋯,0.3346063,0.1517321,0.5024130,0.5900028,8,1293,108,2,37.96,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,Dermatología,37,2021,23,281292,11,Sur Ii,280133,Aranjuez,⋯,0.3346063,0.1517321,0.5024130,0.5900028,5,1501,103,2,23.14,TRUE
HOSPITAL UNIVERSITARIO INFANTA ELENA,Odontoestomatología,0,2021,3,281236,11,Sur Ii,281610,Valdemoro,⋯,0.3346063,0.1517321,0.5024130,0.5900028,0,1240,44,0,0.00,TRUE


## Text data analysis

<font color='tomato'>Select columns</font>

In [148]:
# Select column
text_columns <- sapply(data, is.character)


<font color='darkgrey'><b>Operation</font>

In [149]:
# Analizar datos de texto y verificar su corrección
# e.g. faltas ortografía, etc
# pasar a mayúsculas todas las columnas de texto

data[, text_columns] <- lapply(data[, text_columns], function(x) toupper(x))

## Delete Columns Needless/Irrelevant/Private

<font color='tomato'>Select columns</font>

No aplica


<font color='darkgrey'><b>Operation</font>

In [150]:
# Eliminamos columnas que consideramos irrelevantes o innecesarias
column_names <- colnames(data)
column_names

[1] "Hospital"          "Especialidad"      "total_pacientes"  
 [4] "ano"               "semana"            "CODCNH"           
 [7] "id_area"           "nombre_area"       "cmunicipio"       
[10] "Municipio"         "CAMAS"             "Clase"            
[13] "Dependencia"       "TAC"               "RM"               
[16] "GAM"               "HEM"               "ASD"              
[19] "LIT"               "BCO"               "ALI"              
[22] "SPECT"             "PET"               "MAMOS"            
[25] "DO"                "DIAL"              "X"                
[28] "Y"                 "t3_1"              "t1_1"             
[31] "t2_1"              "t2_2"              "t4_1"             
[34] "t4_2"              "t4_3"              "t5_1"             
[37] "t6_1"              "t7_1"              "t8_1"             
[40] "t9_1"              "t10_1"             "t11_1"            
[43] "t12_1"             "capacidad"         "pacientes"        
[46] "consultas"         "hospitalizaciones" "Target"           
[49] "is_train"

Todas las columnas son relevantes, por lo que no se elimina ninguna.

## Inconsistent Data 

No aplica

<font color='tomato'>Select columns and value</font>

<font color='darkgrey'><b>Operation</font>

## Expected values

## Zeros

No aplica. ETL satisface los requisitos de calidad de los datos para valores cero.

## Single Value

In [151]:
# We obtain the number of different values of each column
distinct_counts <- sapply(data, function(x) n_distinct(x, na.rm = TRUE))
distinct_counts

Hospital      Especialidad   total_pacientes               ano 
               29                16              3528                 3 
           semana            CODCNH           id_area       nombre_area 
               51                29                10                10 
       cmunicipio         Municipio             CAMAS             Clase 
               16                16                28                 3 
      Dependencia               TAC                RM               GAM 
                3                 6                 6                 5 
              HEM               ASD               LIT               BCO 
                4                 4                 2                 1 
              ALI             SPECT               PET             MAMOS 
                5                 5                 2                 5 
               DO              DIAL                 X                 Y 
                3                15                29                29 
             t3_1              t1_1              t2_1              t2_2 
               10                10                10                10 
             t4_1              t4_2              t4_3              t5_1 
               10                10                10                10 
             t6_1              t7_1              t8_1              t9_1 
               10                10                10                10 
            t10_1             t11_1             t12_1         capacidad 
               10                10                10                48 
        pacientes         consultas hospitalizaciones            Target 
              632              4475               322             12635 
         is_train 
                2

In [152]:
# Identify columns with a single unique value
single_value_columns <- which(distinct_counts == 1)
print(single_value_columns)
# Remove columns with a single unique value
data <- data[, -single_value_columns]
colnames(data)


BCO 
 20 


[1] "Hospital"          "Especialidad"      "total_pacientes"  
 [4] "ano"               "semana"            "CODCNH"           
 [7] "id_area"           "nombre_area"       "cmunicipio"       
[10] "Municipio"         "CAMAS"             "Clase"            
[13] "Dependencia"       "TAC"               "RM"               
[16] "GAM"               "HEM"               "ASD"              
[19] "LIT"               "ALI"               "SPECT"            
[22] "PET"               "MAMOS"             "DO"               
[25] "DIAL"              "X"                 "Y"                
[28] "t3_1"              "t1_1"              "t2_1"             
[31] "t2_2"              "t4_1"              "t4_2"             
[34] "t4_3"              "t5_1"              "t6_1"             
[37] "t7_1"              "t8_1"              "t9_1"             
[40] "t10_1"             "t11_1"             "t12_1"            
[43] "capacidad"         "pacientes"         "consultas"        
[46] "hospitalizaciones" "Target"            "is_train"

## Very Few Values

<font color='tomato'>Select rate
</font>

In [153]:
rate <- 0.00005

<font color='darkgrey'><b>Operation</font>

In [154]:
# Summarize columns that have unique values that are less than 
# "porcentage_threshold" percent of the number of rows.


# Show features with over rate rows being the same value
distinct_counts <- sapply(data, function(x) n_distinct(x, na.rm = TRUE))

# Calculate the rate of different values
distinct_rate <- distinct_counts / nrow(data)

# Identify columns with a rate lower than the threshold, excluding 'is_train' and 'Target' columns
columns_to_remove <- names(distinct_rate[distinct_rate < rate & !(names(distinct_rate) %in% c("is_train", "Target"))])

# Print the columns to be removed
print(columns_to_remove)

# Remove columns with a low rate of different values
data <- data[, !(names(data) %in% columns_to_remove)]

[1] "LIT" "PET"


In [155]:
# Summarize the number of unique values in each column
# followed by the percentage of unique values for each 
# variable as a percentage of the total number of rows 
# in the dataset.

# Calculate the number of unique values per column
unique_counts <- sapply(data, function(x) n_distinct(x, na.rm = TRUE))

# Calculate the percentage of unique values for each variable
percentage_unique <- (unique_counts / nrow(data)) * 100

# Create a summary data frame
summary_df <- data.frame(
  Column = names(unique_counts),
  Unique_Count = unique_counts,
  Percentage_Unique = percentage_unique
)

# Print the summary data frame
print(summary_df)

                             Column Unique_Count Percentage_Unique
Hospital                   Hospital           29       0.052083333
Especialidad           Especialidad           16       0.028735632
total_pacientes     total_pacientes         3528       6.336206897
ano                             ano            3       0.005387931
semana                       semana           51       0.091594828
CODCNH                       CODCNH           29       0.052083333
id_area                     id_area           10       0.017959770
nombre_area             nombre_area           10       0.017959770
cmunicipio               cmunicipio           16       0.028735632
Municipio                 Municipio           16       0.028735632
CAMAS                         CAMAS           28       0.050287356
Clase                         Clase            3       0.005387931
Dependencia             Dependencia            3       0.005387931
TAC                             TAC            6       0.01077

<font color='tomato'>Select percent of the number of rows</font>

In [156]:
# Select percent of the number of rows
percentage_threshold_value <- 0.0005

<font color='darkgrey'><b>Operation</font>

In [157]:


# Get the column names to remove
columns_to_remove <- summary_df$Column[summary_df$Percentage_Unique < percentage_threshold_value]
print(columns_to_remove)
# Remove the columns from the data
data <- data[, !(names(data) %in% columns_to_remove)]


character(0)


<font color='darkgrey'><b>Check final columns</font>

In [159]:
# Delete columns that have unique values that are less than 
# "porcentage_threshold" percent of the number of rows.
#
# NOTE: WE MUST EXCLUDE TARGET COLUMN TO DELETE
colnames(data)


[1] "Hospital"          "Especialidad"      "total_pacientes"  
 [4] "ano"               "semana"            "CODCNH"           
 [7] "id_area"           "nombre_area"       "cmunicipio"       
[10] "Municipio"         "CAMAS"             "Clase"            
[13] "Dependencia"       "TAC"               "RM"               
[16] "GAM"               "HEM"               "ASD"              
[19] "ALI"               "SPECT"             "MAMOS"            
[22] "DO"                "DIAL"              "X"                
[25] "Y"                 "t3_1"              "t1_1"             
[28] "t2_1"              "t2_2"              "t4_1"             
[31] "t4_2"              "t4_3"              "t5_1"             
[34] "t6_1"              "t7_1"              "t8_1"             
[37] "t9_1"              "t10_1"             "t11_1"            
[40] "t12_1"             "capacidad"         "pacientes"        
[43] "consultas"         "hospitalizaciones" "Target"           
[46] "is_train"

## Low Variance

**A) Calculating variances**

In [160]:
# Filter numeric columns
numeric_columns <- sapply(data, is.numeric)

# Calculate variance for numeric columns
variance <- sapply(data[, numeric_columns], var, na.rm = TRUE)

# Print the variance values
print(variance)

  total_pacientes               ano            semana            CODCNH 
     7.916352e+05      5.799410e-01      2.232348e+02      2.609947e+05 
          id_area        cmunicipio             CAMAS               TAC 
     1.036880e+01      1.348231e+05      9.577061e+04      1.819295e+00 
               RM               GAM               HEM               ASD 
     1.424520e+00      1.008342e+00      9.465093e-01      8.513827e-01 
              ALI             SPECT             MAMOS                DO 
     1.286587e+00      1.005963e+00      9.369966e-01      4.899018e-01 
             DIAL                 X                 Y              t3_1 
     1.994328e+02      2.441187e-02      1.432827e-02      2.670148e+00 
             t1_1              t2_1              t2_2              t4_1 
     4.643055e+10      1.081162e-04      1.079681e-04      5.199907e-04 
             t4_2              t4_3              t5_1              t6_1 
     1.532418e-04      6.024165e-04      5.255911e-

**B) Automatic calculation and representation of variances**

<font color='tomato'>Define thresholds to check</font>

In [161]:
# define thresholds to check
variance_threshold = 0.00001

**C) Delete variables with low variance**

<font color='tomato'>Select column</font>

<font color='darkgrey'><b>Operation</font>

In [163]:
# drop columns with low variance

# Get the column names with variance lower than the threshold
low_variance_columns <- names(variance[variance < variance_threshold])

# Remove the low variance columns from the dataset
data <- data[, !names(data) %in% low_variance_columns]
colnames(data)

[1] "Hospital"          "Especialidad"      "total_pacientes"  
 [4] "ano"               "semana"            "CODCNH"           
 [7] "id_area"           "nombre_area"       "cmunicipio"       
[10] "Municipio"         "CAMAS"             "Clase"            
[13] "Dependencia"       "TAC"               "RM"               
[16] "GAM"               "HEM"               "ASD"              
[19] "ALI"               "SPECT"             "MAMOS"            
[22] "DO"                "DIAL"              "X"                
[25] "Y"                 "t3_1"              "t1_1"             
[28] "t2_1"              "t2_2"              "t4_1"             
[31] "t4_2"              "t4_3"              "t5_1"             
[34] "t6_1"              "t7_1"              "t8_1"             
[37] "t9_1"              "t10_1"             "t11_1"            
[40] "t12_1"             "capacidad"         "pacientes"        
[43] "consultas"         "hospitalizaciones" "Target"           
[46] "is_train"

## Duplicates

Entendido como ERROR --> Eliminar duplicados

### Identificación de Duplicates

In [166]:
# Remove duplicated rows
data <- data[!duplicated(data), ]

### Eliminación de primera fila duplicada


No aplica

### Eliminación de todas las filas duplicadas

No aplica


### Eliminación de filas duplicadas en columnas concretas

<font color='tomato'>Select columns</font>


No aplica

<font color='darkgrey'><b>Operation</font>


No aplica

## <font color='green'>Data Save</font>

* Solo si se han hecho cambios

* No aplica

<font color='tomato'> Identificamos los datos a guardar</font>

In [168]:
data_to_save <- data


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_06".
* Resto del nombre del archivo de entrada
* Extensión del archivo

Ejemplo: "CU_04_06_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas) y que se ha transformado en el proceso 06

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 09.1

In [33]:
caso <- "CU_25"
proceso <- '_09.1'
tarea <- "_01"
archivo <- "_lista_espera_completo_clean"
proper <- "_v_01"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [ ]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [170]:
file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out <- paste0(oPath, file_save)
write_csv(data_to_save, path_out)

cat('File saved as: ')
path_out

File saved as: 

[1] "Data/Output/CU_25_09.1_01_lista_espera_completo_clean_v_01.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [171]:
path_in <- paste0(iPath, file_save)
file.copy(path_out, path_in, overwrite = TRUE)

[1] TRUE

## <font color='#2874a6'>REPORT</font>

A continuación se realizará un informe de las acciones realizadas

## <font color=' #2874a6 '>Main Actions Carried Out</font>

- Si eran necesarias se han realizado en el proceso 05 por cuestiones de eficiencia

## <font color=' #2874a6 '>Main Conclusions</font>

- Los datos están limpios para el despliegue


## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>